In [1]:
# Install all the dependencies
# Import libraries
# Here we check the heatmap 
# !pip install geopandas
# import geopandas as gpd
# from shapely.geometry import Polygon

import numpy as np
import pandas as pd
import glob
import geopy
import geopy.distance
import datetime
import sys
import folium
from folium import plugins
from folium.plugins import HeatMap


In [2]:
path = r"ship_data_polygon.csv"    # for hosted runtime use this path 


df=pd.read_csv(path,sep=',',low_memory=False)
test_data=df.drop(['Length','Draught','Breadth','SOG','Distance remaining','ETA'], axis=1)
test_data['mmsi'] = test_data['mmsi'].astype(int)

# test_data

In [24]:
# Hamburg polygon
coords_hamburg = [(9.79842, 53.532), (10.05557, 53.42),( 10.10363, 53.53791),( 9.8331 ,53.56137),( 9.79842 ,53.532)]
 
#southern
coords_south = [(5.728244746093775 ,53.76638802023849), 
(5.876560175781277, 53.59231963371792),
(6.079807246093752,53.634682798703146),
(5.9122657421875635,53.81344158746174),
(5.728244746093775,53.76638802023849)]
 
#northern
coords_north = [(5.489292109375013,54.18154401905825), 
(5.527744257812566,54.01888262116618),
(5.733737910156309,54.035016171795), 
(5.670566523437535,54.20243431238705), 
(5.489292109375013, 54.18154401905825)]
 
polygon_geom = Polygon(coords_north)
polygon_geom2 =Polygon(coords_south)
polygon_geom3 =Polygon(coords_hamburg)

crs = {'init': 'epsg:4326'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])  
polygon2 = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom2])
polygon3 = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom3])

print(polygon.geometry)
print(polygon2.geometry)
print(polygon3.geometry)

polygon.to_file(filename='polygon.geojson', driver='GeoJSON')
polygon.to_file(filename='polygon.shp', driver="ESRI Shapefile")
polygon.to_file(filename='polygon2.geojson', driver='GeoJSON')
polygon.to_file(filename='polygon2.shp', driver="ESRI Shapefile")
polygon.to_file(filename='polygon3.geojson', driver='GeoJSON')
polygon.to_file(filename='polygon3.shp', driver="ESRI Shapefile")


0    POLYGON ((5.48929 54.18154, 5.52774 54.01888, ...
Name: geometry, dtype: geometry
0    POLYGON ((5.72824 53.76639, 5.87656 53.59232, ...
Name: geometry, dtype: geometry
0    POLYGON ((9.79842 53.53200, 10.05557 53.42000,...
Name: geometry, dtype: geometry


/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [25]:
# create styles for polygon
style1 = {'fillColor': '#228B22', 'color': '#228B22'}
style2 = {'fillColor': '#00FFFFFF', 'lineColor': '#00FFFFFF'}
# create a map 
m = folium.Map([ 54,7], zoom_start=8, tiles='cartodbpositron')
# add northern polygon to the map
folium.GeoJson(polygon,name='north',style_function=lambda x:style1).add_to(m)  
# add southern polygon to the map
folium.GeoJson(polygon2,name='south',style_function=lambda x:style2).add_to(m) 
# add hamburg polygon to the map 
folium.GeoJson(polygon3).add_to(m)
# pop up lat lon to the map                                               
folium.LatLngPopup().add_to(m)

In [26]:
# look up unique
unique_data=test_data.mmsi.unique
unique_data()

array([209792000, 209857000, 210459000, 209066000, 210925000, 218552000,
       219430000, 218839000, 218816000, 212651000, 218842000, 212464000,
       218025000, 229668000, 235011610, 229384000, 228020700, 228084000,
       235095162, 235076499, 236107000, 235116052, 236112059, 236283000,
       236476000, 236481000, 236691000, 236507000, 244198000, 244267000,
       244338000, 244830098, 244850075, 244768000, 246553000, 245405000,
       247086800, 247341900, 247039400, 246271000, 247385300, 248409000,
       248430000, 248439000, 249207000, 249704000, 255805884, 255623000,
       255801670, 255805557, 255805799, 255805798, 255805699, 255805883,
       255805845, 255805836, 255806032, 256323000, 304868000, 256547000,
       370271000, 266273000, 477711300, 477423600, 477634900, 312256000,
       354135000, 356249000, 357632000, 304013000, 304634000, 304596000,
       305121000, 305063000, 305299000, 305484000, 305837000, 306777000,
       308876000, 373134000, 636092168, 636092361, 

In [27]:
# Here change mmsi number to check the heatmap

## to check the north ship data heatmap uncomment line below

# one_ship=test_data[test_data.mmsi== 209066000] # for north 

## to check the south ship data heatmap uncomment line below

# one_ship=test_data[test_data.mmsi==311000426]   # for south

# to check the entire ship data heatmap uncomment line below
#one_ship=test_data
one_ship=one_ship.drop(['mmsi','bool'], axis=1)

one_ship = one_ship.reindex(['Latitude','Longitude',], axis="columns")
one_ship

,Latitude,Longitude
309518,54.07476,5.515426
309519,54.07518,5.520718
309520,54.07560,5.526138
309521,54.07599,5.531426
309522,54.07638,5.536712
...,...,...
310575,53.55323,9.806743
310576,53.55252,9.810905
310577,53.55183,9.814898
310578,53.55116,9.818884


In [28]:

points = [tuple(x) for x in one_ship.to_numpy()]


ave_lat = sum(p[0] for p in points)/len(points)
ave_lon = sum(p[1] for p in points)/len(points)

# Load map centred on average coordinates
#my_map = folium.Map(location=[ave_lat, ave_lon], zoom_start=5)


#add a heatmap points

HeatMap(points).add_to(m)

m.save("./out.html")

In [29]:
m #south

In [15]:
m # north

In [ ]:
m #south
